In [4]:
#!pip install fastbook

In [3]:
import fastbook
fastbook.setup_book()
from fastbook import *
from IPython.display import display,HTML

from fastai.text.all import *

/jet/var/python/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/jet/var/python/lib/python3.6/site-packages/fastbook/__init__.py:22: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


Materials ADW_LSTM

https://docs.fast.ai/tutorial.datablock

https://humboldt-wi.github.io/blog/research/information_systems_1819/group4_ulmfit/

https://gluon-nlp.mxnet.io/examples/language_model/language_model.html

https://rajeethkumardharmaraj.wordpress.com/2018/08/26/text-classification-with-pytorch-and-fastai-explanation-part-2/

https://blog.usejournal.com/finding-data-block-nirvana-a-journey-through-the-fastai-data-block-api-c38210537fe4

In [2]:

path = untar_data(URLs.IMDB)

In [3]:
path.ls()

(#7) [Path('/jet/.fastai/data/imdb/unsup'),Path('/jet/.fastai/data/imdb/imdb.vocab'),Path('/jet/.fastai/data/imdb/tmp_lm'),Path('/jet/.fastai/data/imdb/tmp_clas'),Path('/jet/.fastai/data/imdb/test'),Path('/jet/.fastai/data/imdb/README'),Path('/jet/.fastai/data/imdb/train')]

- Tokenization:: Convert the text into a list of words (or characters, or substrings, depending on the granularity of your model)
- Numericalization:: Make a list of all of the unique words that appear (the vocab), and convert each word into a number, by looking up its index in the vocab
- Language model data loader creation:: fastai provides an LMDataLoader class which automatically handles creating a dependent variable that is offset from the independent variable by one token. It also handles some important details, such as how to shuffle the training data in such a way that the dependent and independent variables maintain their structure as required
- Language model creation:: We need a special kind of model that does something we haven't seen before: handles input lists which could be arbitrarily big or small. There are a number of ways to do this; in this chapter we will be using a recurrent neural network (RNN). We will get to the details of these RNNs in the 

### Word Tokenization with fastai

In [4]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [5]:
txt = files[0].open().read(); txt[:75]

'Charles Bronson plays a liberal, mild-mannered architect. His wife (Hope La'

As we write this book, the default English word tokenizer for fastai uses a library called spaCy. It has a sophisticated rules engine with special rules for URLs, individual special English words, and much more. Rather than directly using SpacyTokenizer, however, we'll use WordTokenizer, since that will always point to fastai's current default word tokenizer (which may not necessarily be spaCy, depending when you're reading this).

Let's try it out. We'll use fastai's coll_repr(collection, n) function to display the results. This displays the first n items of collection, along with the full size—it's what L uses by default. Note that fastai's tokenizers take a collection of documents to tokenize, so we have to wrap txt in a list:

In [6]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#272) ['Charles','Bronson','plays','a','liberal',',','mild','-','mannered','architect','.','His','wife','(','Hope','Lange',')','is','murdered','and','his','daughter','is','gang','-','raped','into','a','near','-'...]


In [7]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [8]:
next(iter(spacy(['The U.S. dollar $1 is $1.00.'])))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [9]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#309) ['xxbos','xxmaj','charles','xxmaj','bronson','plays','a','liberal',',','mild','-','mannered','architect','.','xxmaj','his','wife','(','hope','xxmaj','lange',')','is','murdered','and','his','daughter','is','gang','-','raped'...]


Notice that there are now some tokens that start with the characters "xx", which is not a common word prefix in English. These are special tokens.

For example, the first item in the list, xxbos, is a special token that indicates the start of a new text ("BOS" is a standard NLP acronym that means "beginning of stream"). By recognizing this start token, the model will be able to learn it needs to "forget" what was said previously and focus on upcoming words.

These special tokens don't come from spaCy directly. They are there because fastai adds them by default, by applying a number of rules when processing text. These rules are designed to make it easier for a model to recognize the important parts of a sentence. In a sense, we are translating the original English language sequence into a simplified tokenized language—a language that is designed to be easy for a model to learn.

For instance, the rules will replace a sequence of four exclamation points with a single exclamation point, followed by a special repeated character token, and then the number four. In this way, the model's embedding matrix can encode information about general concepts such as repeated punctuation rather than requiring a separate token for every number of repetitions of every punctuation mark. Similarly, a capitalized word will be replaced with a special capitalization token, followed by the lowercase version of the word. This way, the embedding matrix only needs the lowercase versions of the words, saving compute and memory resources, but can still learn the concept of capitalization.

Here are some of the main special tokens you'll see:

- xxbos:: Indicates the beginning of a text (here, a review)
- xxmaj:: Indicates the next word begins with a capital (since we lowercased everything)
- xxunk:: Indicates the next word is unknown

To see the rules that were used, you can check the default rules:

In [10]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [11]:
??replace_rep

In [12]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

### Subword Tokenization

In [13]:
txts = L(o.open().read() for o in files[:2000])

In [14]:
txts[0]

"Charles Bronson plays a liberal, mild-mannered architect. His wife (Hope Lange) is murdered and his daughter is gang-raped into a near-comatose state by a gang of thugs (one played by then unknown Jeff Goldblum!). He slowly becomes a vigilante--going out late at night and shooting and killing any muggers and thieves that attack him. Somewhat unrealistically he's attacked every single night he goes out--was NY THAT bad in 1974? He becomes a media hero and police detective Vincent Gardenia is assigned to stop him.<br /><br />Interesting movie--on one hand you're all for Bronson and, on the other, you're disgusted with what he's doing. Both sides of the issue are presented but not dealt with in much depth--this IS a Charles Bronson movie after all! Film is dated (NY has gotten much safer since this came out) but is still powerful.<br /><br />Bronson gives a bad performance which somewhat diminishes the films impact, but everybody else is great--especially Lange and Gardenia. And look for

In addition to the word tokenization approach seen in the last section, another popular tokenization method is subword tokenization. Word tokenization relies on an assumption that spaces provide a useful separation of components of meaning in a sentence. However, this assumption is not always appropriate. For instance, consider this sentence: 我的名字是郝杰瑞 ("My name is Jeremy Howard" in Chinese). That's not going to work very well with a word tokenizer, because there are no spaces in it! Languages like Chinese and Japanese don't use spaces, and in fact they don't even have a well-defined concept of a "word." There are also languages, like Turkish and Hungarian, that can add many subwords together without spaces, creating very long words that include a lot of separate pieces of information.

To handle these cases, it's generally best to use subword tokenization. This proceeds in two steps:

Analyze a corpus of documents to find the most commonly occurring groups of letters. These become the vocab.
Tokenize the corpus using this vocab of subword units.
Let's look at an example. For our corpus, we'll use the first 2,000 movie reviews:

We instantiate our tokenizer, passing in the size of the vocab we want to create, and then we need to "train" it. That is, we need to have it read our documents and find the common sequences of characters to create the vocab. This is done with setup. As we'll see shortly, setup is a special fastai method that is called automatically in our usual data processing pipelines. Since we're doing everything manually at the moment, however, we have to call it ourselves. Here's a function that does these steps for a given vocab size, and shows an example output:

In [15]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [16]:
subword(1000)

'▁C h ar le s ▁B ro n son ▁play s ▁a ▁li ber al , ▁ m il d - man ner ed ▁a r ch it e c t . ▁H is ▁wife ▁( H op e ▁La'

When using fastai's subword tokenizer, the special character ▁ represents a space character in the original text.

If we use a smaller vocab, then each token will represent fewer characters, and it will take more tokens to represent a sentence:

In [17]:
subword(300)

'▁C h ar le s ▁B ro n s on ▁p la y s ▁a ▁ li b er al , ▁ m il d - m an n er ed ▁ ar ch it e c t . ▁H'

Picking a subword vocab size represents a compromise: a larger vocab means fewer tokens per sentence, which means faster training, less memory, and less state for the model to remember; but on the downside, it means larger embedding matrices, which require more data to learn.

Overall, subword tokenization provides a way to easily scale between character tokenization (i.e., using a small subword vocab) and word tokenization (i.e., using a large subword vocab), and handles every human language without needing language-specific algorithms to be developed. It can even handle other "languages" such as genomic sequences or MIDI music notation! For this reason, in the last year its popularity has soared, and it seems likely to become the most common tokenization approach (it may well already be, by the time you read this!).

### Numericalization with fastai

Numericalization is the process of mapping tokens to integers. The steps are basically identical to those necessary to create a Category variable, such as the dependent variable of digits in MNIST:

- Make a list of all possible levels of that categorical variable (the vocab).
- Replace each level with its index in the vocab.

In [18]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#309) ['xxbos','xxmaj','charles','xxmaj','bronson','plays','a','liberal',',','mild','-','mannered','architect','.','xxmaj','his','wife','(','hope','xxmaj','lange',')','is','murdered','and','his','daughter','is','gang','-','raped'...]


In [19]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#309) ['xxbos','xxmaj','charles','xxmaj','bronson','plays','a','liberal',',','mild'...]

In [20]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2112) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','it','in','i'...]"

Our special rules tokens appear first, and then every word appears once, in frequency order. 

The defaults to Numericalize are min_freq=3,max_vocab=60000. max_vocab=60000 results in fastai replacing all words other than the most common 60,000 with a special unknown word token, xxunk. This is useful to avoid having an overly large embedding matrix, since that can slow down training and use up too much memory, and can also mean that there isn't enough data to train useful representations for rare words. 

However, this last issue is better handled by setting min_freq; the default min_freq=3 means that any word appearing less than three times is replaced with xxunk.

fastai can also numericalize your dataset using a vocab that you provide, by passing a list of words as the vocab parameter.

Once we've created our Numericalize object, we can use it as if it were a function:

In [21]:
nums = num(toks)[:20]; nums

TensorText([   2,    8, 1022,    8, 1231,  245,   13,    0,   10,    0,   24,    0,    0,   11,    8,   42,  299,   34,  461,    8])

In [22]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj charles xxmaj bronson plays a xxunk , xxunk - xxunk xxunk . xxmaj his wife ( hope xxmaj'

### Putting Our Texts into Batches for a Language Model

In [23]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
print(len(stream.split()))
tokens = tkn(stream)
print(len(tokens))
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

76
90


xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


Going back to our previous example with 6 batches of length 15, if we chose a sequence length of 5, that would mean we first feed the following array:

In [24]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [25]:
#  then this one
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [26]:
# And finally:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [27]:
nums200 = toks200.map(num)

In [28]:
# and then passing that to LMDataLoader:
dl = LMDataLoader(nums200)  # by default bs=64 and seq_len=72

In [29]:
x,y = first(dl)
x.shape,y.shape

((64, 72), (64, 72))

In [30]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj charles xxmaj bronson plays a xxunk , xxunk - xxunk xxunk . xxmaj his wife ( hope xxmaj'

In [31]:
# The dependent variable is the same thing offset by one token:

In [32]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj charles xxmaj bronson plays a xxunk , xxunk - xxunk xxunk . xxmaj his wife ( hope xxmaj xxunk'

### Training a Text Classifier

Training a Text Classifier
As we saw at the beginning of this chapter, there are two steps to training a state-of-the-art text classifier using transfer learning: first we need to fine-tune our language model pretrained on Wikipedia to the corpus of IMDb reviews, and then we can use that model to train a classifier.

As usual, let's start with assembling our data.

fastai handles tokenization and numericalization automatically when TextBlock is passed to DataBlock. All of the arguments that can be passed to Tokenize and Numericalize can also be passed to TextBlock. In the next chapter we'll discuss the easiest ways to run each of these steps separately, to ease debugging—but you can always just debug by running them manually on a subset of your data as shown in the previous sections. And don't forget about DataBlock's handy summary method, which is very useful for debugging data issues.

Here's how we use TextBlock to create a language model, using fastai's defaults:

In [33]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),  # min_freq=3 from Numericalize class may be added
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [34]:
get_text_files(path=path,folders='train')[1]

Path('/jet/.fastai/data/imdb/train/pos/11915_10.txt')

One thing that's different to previous types we've used in DataBlock is that we're not just using the class directly (i.e., TextBlock(...), but instead are calling a class method. A class method is a Python method that, as the name suggests, belongs to a class rather than an object. (Be sure to search online for more information about class methods if you're not familiar with them, since they're commonly used in many Python libraries and applications; we've used them a few times previously in the book, but haven't called attention to them.) The reason that TextBlock is special is that setting up the numericalizer's vocab can take a long time (we have to read and tokenize every document to get the vocab). To be as efficient as possible it performs a few optimizations:

- It saves the tokenized documents in a temporary folder, so it doesn't have to tokenize them more than once
- It runs multiple tokenization processes in parallel, to take advantage of your computer's CPUs
We need to tell TextBlock how to access the texts, so that it can do this initial preprocessing—that's what from_folder does.

show_batch then works in the usual way:

In [35]:
dls_lm.show_batch(max_n=3)

,text,text_
0,"xxbos xxmaj this reworking of xxmaj anthony xxmaj shaffer 's classic play did not last long in cinemas . xxmaj having recently suffered through it on cable , i still congratulate myself for not wasting money on a ticket . xxmaj director xxmaj kenneth xxmaj branagh , writer xxmaj harold xxmaj pinter , and star / producer xxmaj jude xxmaj law deluded themselves that their prestige alone could sustain this travesty through an interminable 93 minutes , without the fun","xxmaj this reworking of xxmaj anthony xxmaj shaffer 's classic play did not last long in cinemas . xxmaj having recently suffered through it on cable , i still congratulate myself for not wasting money on a ticket . xxmaj director xxmaj kenneth xxmaj branagh , writer xxmaj harold xxmaj pinter , and star / producer xxmaj jude xxmaj law deluded themselves that their prestige alone could sustain this travesty through an interminable 93 minutes , without the fun or"
1,"is coming from a fan ! xxbos xxmaj well xxup ok , i watched this movie a little over 2 years ago and now i pulled it out of the dusty shelf to watch it again and i must say , i actually think this movie is good . xxmaj this movie caught a buzz as the bootleg i xxmaj know xxmaj what xxmaj you xxmaj did xxmaj last xxmaj summer 3 , much like xxmaj final xxmaj stab was","coming from a fan ! xxbos xxmaj well xxup ok , i watched this movie a little over 2 years ago and now i pulled it out of the dusty shelf to watch it again and i must say , i actually think this movie is good . xxmaj this movie caught a buzz as the bootleg i xxmaj know xxmaj what xxmaj you xxmaj did xxmaj last xxmaj summer 3 , much like xxmaj final xxmaj stab was to"
2,"xxmaj kong destroy xxmaj peter xxmaj jackson 's of this year . xxbos xxmaj the tweedy professor - types thought they had it all figured out . xxmaj today 's peoples who inhabit xxmaj polynesia descended from xxunk xxmaj asians , xxunk moving from the xxmaj far xxmaj east , island to island , xxunk into xxmaj tahiti and all the other exotic tropic isles of the xxmaj south xxmaj pacific over thousands of years . xxmaj but the established","kong destroy xxmaj peter xxmaj jackson 's of this year . xxbos xxmaj the tweedy professor - types thought they had it all figured out . xxmaj today 's peoples who inhabit xxmaj polynesia descended from xxunk xxmaj asians , xxunk moving from the xxmaj far xxmaj east , island to island , xxunk into xxmaj tahiti and all the other exotic tropic isles of the xxmaj south xxmaj pacific over thousands of years . xxmaj but the established thinking"


In [36]:
dls_lm.vocab[-10]

'juhee'

In [37]:
len(dls_lm.vocab), len(set((dls_lm.vocab)))

(60008, 60001)

Now that our data is ready, we can fine-tune the pretrained language model.

### Fine-Tuning the Language Model
To convert the integer word indices into activations that we can use for our neural network, we will use embeddings, just like we did for collaborative filtering and tabular modeling. Then we'll feed those embeddings into a recurrent neural network (RNN), using an architecture called AWD-LSTM (we will show you how to write such a model from scratch in

In [34]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()])  #.to_fp16()   if GPU is active

In [35]:
learn.fit_one_cycle(1, 2e-2)

RuntimeError: The size of tensor a (400) must match the size of tensor b (0) at non-singleton dimension 2

In [39]:
??learn.predict

### Saving and Loading Models
This will create a file in learn.path/models/ named 1epoch.pth. If you want to load your model in another machine after creating your Learner the same way, or resume training later, you can load the content of this file with:

In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')

Once the initial training has completed, we can continue fine-tuning the model after unfreezing:

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

Once this is done, we save all of our model except the final layer that converts activations to probabilities of picking each token in our vocabulary. The model not including the final layer is called the encoder. We can save it with save_encoder:

In [ ]:
learn.save_encoder('finetuned')

 - jargon: Encoder: The model not including the task-specific final layer(s). This term means much the same thing as body when applied to vision CNNs, but "encoder" tends to be more used for NLP and generative models.

This completes the second stage of the text classification process: fine-tuning the language model. We can now use it to fine-tune a classifier using the IMDb sentiment labels.

### Text Generation

Before we move on to fine-tuning the classifier, let's quickly try something different: using our model to generate random reviews. Since it's trained to guess what the next word of the sentence is, we can use the model to write new reviews:

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

As you can see, we add some randomness (we pick a random word based on the probabilities returned by the model) so we don't get exactly the same review twice. Our model doesn't have any programmed knowledge of the structure of a sentence or grammar rules, yet it has clearly learned a lot about English sentences: we can see it capitalizes properly (I is just transformed to i because our rules require two characters or more to consider a word as capitalized, so it's normal to see it lowercased) and is using consistent tense. The general review makes sense at first glance, and it's only if you read carefully that you can notice something is a bit off. Not bad for a model trained in a couple of hours!

But our end goal wasn't to train a model to generate reviews, but to classify them... so let's use this model to do just that.

### Creating the Classifier DataLoaders

We're now moving from language model fine-tuning to classifier fine-tuning. To recap, a language model predicts the next word of a document, so it doesn't need any external labels. A classifier, however, predicts some external label—in the case of IMDb, it's the sentiment of a document.

This means that the structure of our DataBlock for NLP classification will look very familiar. It's actually nearly the same as we've seen for the many image classification datasets we've worked with:

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)